In [26]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
from scipy.sparse import load_npz, save_npz, csr_matrix
import joblib

# project root & import path
PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT / "src"))


from Feature_pipeline import TabularFeatureReducer, FingerprintReducer


In [27]:
DATA_PROC = PROJECT_ROOT / "data" / "processed"

# train
X_train = load_npz(DATA_PROC / "X_train_default.npz")
y_train = np.load(DATA_PROC / "y_train_default.npy")
train_meta = pd.read_parquet(DATA_PROC / "meta_train_default.parquet")

# test
X_test = load_npz(DATA_PROC / "X_test_default.npz")
test_meta = pd.read_parquet(DATA_PROC / "meta_test_default.parquet")

# sparse -> dense for the reducer
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

tab_reducer = TabularFeatureReducer(
    clip_z=5.0,
    corr_threshold=0.95,
    top_n=300,
)

X_train_red = tab_reducer.fit_transform(X_train_dense, y_train)
X_test_red = tab_reducer.transform(X_test_dense)


c:\Users\hisuk\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\hisuk\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018180 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3386
[LightGBM] [Info] Number of data points in the train set: 19616, number of used features: 616
[LightGBM] [Info] Start training from score 382.867786


In [28]:
# save reduced features as sparse (to save space)
X_train_red_csr = csr_matrix(X_train_red)
X_test_red_csr = csr_matrix(X_test_red)

save_npz(DATA_PROC / "X_train_tabred_default.npz", X_train_red_csr)
save_npz(DATA_PROC / "X_test_tabred_default.npz", X_test_red_csr)

# save the fitted reducer for later inference
joblib.dump(tab_reducer, DATA_PROC / "tabular_reducer_default.pkl")

print("Saved:")
print(" - X_train_tabred_default.npz")
print(" - X_test_tabred_default.npz")
print(" - tabular_reducer_default.pkl")


Saved:
 - X_train_tabred_default.npz
 - X_test_tabred_default.npz
 - tabular_reducer_default.pkl
